# Combining models

There are many methods to combine models. 

🗳️ **Voting**

Just count what each models predicts and takes the most common prediction. This is simple to use but has it pitfalls.
- **Strong correlation**: Sometimes two models have almost the same outcome, with voting you just double their influence.
- **Repeat mistakes**: Not every model will be great for the same kind of data. If most of the models make a same mistake on a datapoint it does not mean they are right.

🗳️ + 🔢 **Voting with scores**

To counter these problems we need models to differ in prediction and have a confidence score so we know when to listen.
In my model with Umap+nearest neighbors it was very rare for correct predictions to have an ambiguity in there neighbors.

## Correlation

Lets find a model that does NOT correlate strongly with my model. For this I am looking for notebooks that introduce their own solution not just take existing submissions and combining them.

In [8]:
all_submisions_paths = Path("/kaggle/input/").glob("**/submission.csv")
for i in all_submisions_paths:
    print(i.name, i)

submission.csv /kaggle/input/truthgpt-spotting-real-in-this-fake-world/submission.csv
submission.csv /kaggle/input/extracting-features-with-spacy/submission.csv
submission.csv /kaggle/input/0-84232-enssenbel-4-model-impostor-hunt/submission.csv
submission.csv /kaggle/input/0-89211-fake-or-real-distil-bert-self-attention/submission.csv
submission.csv /kaggle/input/dimension-reduction-with-umap/submission.csv
submission.csv /kaggle/input/0-87759-fake-or-real-bert-pca-randomforest/submission.csv
submission.csv /kaggle/input/memory-optimized-transformers-for-impostor-hunt/submission.csv
